In [278]:
import numpy as np
import os
from os import listdir
import librosa as lb
import pandas as pd
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import glob
import random
import torch

In [279]:
if os.path.exists('data')==False:
    os.mkdir('data')

## Making a Experiment Folder

In [280]:
exp = 'exp-12'

In [281]:
if os.path.exists(os.path.join('data',exp))==False:
    os.mkdir(os.path.join('data',exp))
else:
    print('Folder is exists')

Folder is exists


In [282]:
dataset = 'dataset/Merge-Data/'

## Noise Augmentation

In [283]:
noisetypes = ['noise']
musan_path = 'dataset/Musan-Data'

noisesnr   = {'noise':[0,15],'speech':[13,20],'music':[5,15]}
numnoise   = {'noise':[1,1], 'speech':[3,7],  'music':[1,1] }
noiselist  = {}
augment_files   = glob.glob(os.path.join(musan_path,'*/*/*.wav'));

for file in augment_files:
    if not file.split('/')[-3] in noiselist:
        noiselist[file.split('/')[-3]] = []
    noiselist[file.split('/')[-3]].append(file)

In [284]:
def additive_noise(noisetypes, noiselist, noisesnr, numnoise, audio, noisecat=None):
    if noisecat is None:
        noisecat = noisetypes[0]

    wav = audio.reshape(audio.shape[-1])
    clean_db = 10 * np.log10(np.mean(wav ** 2)+1e-4) 
    numnoise = numnoise[noisecat]
    noiselist = random.sample(noiselist[noisecat], random.randint(numnoise[0],numnoise[1]))

    final_noise_audio = 0
    for noise in noiselist:
        noise_wav,sr = lb.load(noise)
        noise_wav = noise_wav.reshape(noise_wav.shape[-1])
        noise_wav_len = noise_wav.shape[0]
        wav_len = wav.shape[0]
        if noise_wav_len <= wav_len:
            continue

        noise_start_slice = random.randint(0,noise_wav_len-(wav_len+1))
        noise_wav = noise_wav[noise_start_slice:noise_start_slice+wav_len]

        noise_snr = random.uniform(noisesnr[noisecat][0],noisesnr[noisecat][1])
        noise_db = 10 * np.log10(np.mean(noise_wav ** 2)+1e-4)
        final_noise_audio = np.sqrt(10 ** ((clean_db - noise_db - noise_snr) / 10)) * noise_wav

    out_audio = np.array(wav + final_noise_audio)
    out_audio = out_audio.reshape(-1)
    return out_audio

## Normalization

In [285]:
def normalization(sound,sr,scaler):
    stft = np.abs(lb.stft(sound))

    mfccs = np.mean(lb.feature.mfcc(y=sound, sr=sample_rate, n_mfcc=40),axis=1)
    mfccs_norm = scaler.fit_transform(np.reshape(mfccs,[mfccs.shape[0],1]))

    chroma = np.mean(lb.feature.chroma_stft(S=stft, sr=sample_rate),axis=1)
    chroma_norm = scaler.fit_transform(np.reshape(chroma,[chroma.shape[0],1]))

    mel = np.mean(lb.feature.melspectrogram(sound, sr=sample_rate),axis=1)
    mel_norm = scaler.fit_transform(np.reshape(mel,[mel.shape[0],1]))

    contrast = np.mean(lb.feature.spectral_contrast(S=stft, sr=sample_rate),axis=1)
    contrast_norm = scaler.fit_transform(np.reshape(contrast,[contrast.shape[0],1]))

    tonnetz = np.mean(lb.feature.tonnetz(y=lb.effects.harmonic(sound), sr=sample_rate),axis=1)
    tonnetz_norm = scaler.fit_transform(np.reshape(tonnetz,[tonnetz.shape[0],1]))

    concat = np.concatenate((mfccs,chroma,mel,contrast,tonnetz))
    concat_norm = np.concatenate((mfccs_norm,chroma_norm,mel_norm,contrast_norm,tonnetz_norm))
    
    return concat,concat_norm

In [286]:
scaler = StandardScaler()

## Extracting Train Data Feature

In [287]:
df_train = pd.read_csv('csv_files/experiment_data/train.csv')
df_train.head()

,filename,label
0,aHJPsm6esHeW5apkd8KAFdia9zj2.wav,positive
1,ptuwF1pj4ibIoVuZmt4nKb5naiZ2.wav,positive
2,EbWP0XeMpnbrTsljro0dOGKIell1_heavy.wav,positive
3,Ued1DKV2NkUnzWjJcPMxA6Q261r1_heavy.wav,positive
4,ZaeW7vuikQgf7984JVqXYOtUQMs2.wav,positive


In [288]:
img_train = []
label_train = []

to_hot_one = {'positive':0, 'negative':1}

for index,row in df_train.iterrows():
    name = row['filename']
    label = row['label']
    filename = os.path.join(dataset,name)
    
    try:
        print(name, label)
        sound, sr = lb.load(filename, sr=None)
        sound = lb.util.pad_center(sound, 30*sr)
        sound_aug = additive_noise(noisetypes, noiselist, noisesnr, numnoise, audio=sound, noisecat=None)
        
        concat,concat_norm = normalization(sound,sr,scaler)
        img_train.append(concat_norm)
        label_train.append(to_hot_one[label])
        
        concat_aug,concat_aug_norm = normalization(sound_aug,sr,scaler)
        img_train.append(concat_aug_norm)
        label_train.append(to_hot_one[label])        

    except:
        print('None')

aHJPsm6esHeW5apkd8KAFdia9zj2.wav positive
ptuwF1pj4ibIoVuZmt4nKb5naiZ2.wav positive
EbWP0XeMpnbrTsljro0dOGKIell1_heavy.wav positive
Ued1DKV2NkUnzWjJcPMxA6Q261r1_heavy.wav positive
ZaeW7vuikQgf7984JVqXYOtUQMs2.wav positive
gDBsK8LcY0Om4DujIs4lFSsktWu1_heavy.wav positive
M9CdGPOCUFPWV7KnLgvStPgjLrF3_heavy.wav positive
GBwlY1I67YdQ47ceNFdpK87kEr33.wav positive
RmWwhKI8kjPi5qN9H740rfm0jHA2_heavy.wav positive
Fi2lLmV3LHR5brvXs0z1JSKDvnI2.wav positive
8JuoYlwoiUgaSSUUKQnB7F7fG5S2_heavy.wav positive
KFQ0ksPwkDcLUUjbVBYWGWr9ei62_heavy.wav positive
DRoZujDy6CWVFL0UefbHEgCcD2l2.wav positive
o8LJfYAChnd5Jv0WVa6Ni7jpHf92.wav positive
75OYElXmxdQTQne9breAIqnSo7Z2_heavy.wav positive
rB4ewmXSc9P5IPhLffnBasjYU1s2_heavy.wav positive
NVzhTN4CXebH2lQVFkBTyl6ue5h2_heavy.wav positive
BVcBPtxcv3cqEdIkhjM6YenlWtb2.wav positive
46bIeJk9J8XrD4ND2IPRMQzjJBt2_heavy.wav positive
HK3FfcI5SyXc7CtQMaQiMQ2ia1n2_heavy.wav positive
saaw7uHQJQesN9AmgKyXNFAaXB82.wav positive
etg5L3iyQ3fQAyyn4uZ9jYuoUf13_heavy.wav positiv

/home/viblab/.local/lib/python3.9/site-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")
/home/viblab/.local/lib/python3.9/site-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")
/home/viblab/.local/lib/python3.9/site-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")
/home/viblab/.local/lib/python3.9/site-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")


AWMx6dOBlFPdjWa86wkLjsGZ6by2.wav positive
HK3FfcI5SyXc7CtQMaQiMQ2ia1n2.wav positive
6ZovArshO1MTe2tpxFWrHmORa2R2_heavy.wav positive
FC4RKq4W8uRrjmin8n2BXC4SrnU2.wav positive
zbIXdZwHjgb2uDtKqIgcSnr7pyD2.wav positive
x89c2X4Q5LdiRvU9XWVkU1XXLE13.wav positive
S0ZhYGoCVfXWMHzPGZ41PvHYiw12_heavy.wav positive
4bnHQCj8tnSEx37AR5HiQAH2vB32_heavy.wav positive
M43FaHpw1ScIi97eG6LBx2fBaJ92.wav positive
IimA0GnOm7ZR4nEr8k0kG4eaR7A3_heavy.wav positive
d7w3B2YcJ3TLx58ryhiASEtwaAu1.wav positive
ffrNAGEUQEfL39VAP9Eau1a3UUX2.wav positive
a96ovf71KqfbVIG8ZF5m6DaRE9Z2.wav positive
0Js6ZUZQ9NUnu568Fh7B6mZ1R8o1_heavy.wav positive


/home/viblab/.local/lib/python3.9/site-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")
/home/viblab/.local/lib/python3.9/site-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")


saaw7uHQJQesN9AmgKyXNFAaXB82_heavy.wav positive
8Vz9jwmadkRfjbwrHwGZ2MsUtcz1_heavy.wav positive
nSVDxo9HNmPPggf6WSuAGg9XU6p1_heavy.wav positive
Y7RB11Ha8VNbBDjGoYVuIxnpYKA2_heavy.wav positive
fX1IzMBBKwXISMTgnZUFQa6o5Rh2_heavy.wav positive
31euepHD0deCxTd2nJ1wzXCk5EF3_heavy.wav positive
YHHZYBxPHsUWYITjtMRAixs3DX42.wav positive
YE3Hs5yD9BeIzT8cctZEBKGJIXN2_heavy.wav positive


...

46ab32b1-3748-4b01-853a-0f4243d089f8.wav positive
4cfe71ea-00f2-4595-9903-85dd74968690.wav positive
5d80da79-cf2d-4a6e-9adb-1846cc3b502c.wav positive
dc245a1e-cce1-47f4-914f-e4fa8dbd9917.wav positive
c981bef7-8d7b-4f52-a6ef-f0368edb031e.wav positive
5d9c941a-089f-4c65-b692-5a1bf0d091a4.wav positive
a7ddfa23-7296-40b5-8ab0-c44be913f5dd.wav positive
4934dd77-80e0-42b9-bfc4-995b03b76450.wav positive
f114a26d-73cc-4065-814b-4de83dbda99b.wav positive
d85d1969-896c-47ba-9f9b-540e97f922c6.wav positive
37fdf4e5-d6f0-431d-a198-28dae66a4e84.wav positive
6868da77-a36a-4fb4-afa5-aba26bd2bbd5.wav positive
88184a46-aac5-404c-9477-bcf62d5d9097.wav positive
d636f361-7528-43a9-ba25-e0db9b8ff97c.wav positive
f64f33a5-b6e1-4d5c-ab1b-8d48dd826440.wav positive
bd14922a-f88e-4b99-a338-360789a40b69.wav positive
ab8ecc44-d33a-4e85-8e31-28cfe827fd17.wav positive
9aea1b03-9fc0-48c9-9e62-a6ac15633c03.wav positive
195f9381-cf35-45da-b0ae-cc4ed04e9283.wav positive
dd87bf69-1802-47aa-9586-b840d685797a.wav positive


2475fcc6-74d7-46b8-b35b-4f134d7d57bc.wav positive
fdbabb79-c296-4b93-9e02-06b290916fe7.wav positive
244757ee-75ad-4cdd-938e-a0646aaf4290.wav positive
7ae1ffe1-2259-411f-8ead-6c107e01e824.wav positive
913c4ae7-9245-458d-9ea2-6a2fea6042f8.wav positive
e2e3d4e1-3169-4f8c-a5ad-f157d97c4a40.wav positive
0c181c0d-ea12-4b3c-9356-9e41f9d62a8e.wav positive
80e4f0e6-37ab-4eba-831b-f139ef7f47a4.wav positive
774bafb8-5d1c-42c5-a0a8-91ba0a85a04b.wav positive
22fdf3c1-34b8-411f-a757-4b8cc642d190.wav positive
1cdc0a4b-9521-4b2d-a8b7-ee708232f594.wav positive
d00123d0-397f-43b1-b5d6-bf4584451214.wav positive
6ac1cde9-7727-49d2-848c-983e2e541845.wav positive
18c36541-7566-4f46-8264-3716e2e71f30.wav positive
2d40f13a-02a0-44b4-8d85-79c129c181e2.wav positive
ce946dfd-6e95-4b5a-be31-480c61e670f5.wav positive
c55e4ac5-03f1-4432-ada6-be2c30e76bcb.wav positive
3d89ee73-6ff9-44bf-9e0c-7c51bdfd7ce6.wav positive
6ea6af0c-0f77-46d6-a65b-8df3a925d231.wav positive
d299cadd-d426-4d7c-ac8d-037e25fa2c9a.wav positive


bf7f0dd3-eaef-4912-8edc-c919fd98df62.wav positive
56ff4794-4d52-4b89-8e9c-68ab3d8d1a48.wav positive
3e4f0ebc-7100-48a8-b164-4d90b8dee8db.wav positive
983a0312-f88b-4a37-b632-d9dd5cd29407.wav positive
4c2840ed-fd4b-4a86-b976-5cc040d6ea4e.wav positive
aa059172-1850-4afe-8809-2fe79603286a.wav positive
cd9de27d-87fa-43e6-a452-8cdaba3eae4c.wav positive
6dc3c03a-aeaa-4130-8295-c09e80895c76.wav positive
46f9a3c5-5610-462c-b6b4-ecc9c7dd105a.wav positive
7a985f89-ed5d-41e2-9f5d-8d8982508665.wav positive
760cb050-4175-4498-817d-a9976e359c52.wav positive
080b6119-f1ab-4337-93dd-aaccf73b4ee1.wav positive
bfb9c86e-eead-44ac-8969-ea42e869a3a3.wav positive
76779311-1f9c-40b5-845d-bde19b0d9c0e.wav positive
5ad7b5d3-e9b3-457a-8946-39bd3e1fe3df.wav positive
abbff660-5d13-4c31-b842-bbf2189ff14e.wav positive
01bf156f-a8ff-48f6-9335-8a1f5bb4c1b7.wav positive
db214c02-4dab-4756-bf08-cca015ed8eff.wav positive
811b0a98-9b54-4fd9-95a9-6722383e4100.wav positive
2a9b4fb5-9073-4695-a812-20bce3bb40e1.wav positive


7a222f35-cc21-4085-8202-059ef3ee15d3.wav positive
f9bbf4dd-6255-444b-bf5c-c33c11e7c350.wav positive
1fd4bd15-88c4-47ae-bbe3-2434af431df5.wav positive
5648bc0c-24f9-430c-bdbe-8f81522b8fc3.wav positive
4a5cdc39-a64d-4df0-b9d9-42bb4683c6bb.wav positive
5ff1afa8-c3d7-4c6e-a888-0e99948deeea.wav positive
d26f6631-4e6b-4961-bb81-f6bd80fe3eff.wav positive
91678983-1d96-4390-8f09-55df6a75bb36.wav positive
28d4e487-0d9d-4911-951d-5de7fcb5c986.wav positive
7d2e208f-4b40-43c8-8f85-e05f9671db2c.wav positive
11371e2c-f0df-4805-baf5-0753ade8261c.wav positive
892ba9c4-bc94-49b2-97ce-01fa057a5386.wav positive
a58896fb-dd00-46e0-a2a6-c2d25a9f13e9.wav positive
a5db38cf-708c-4982-b43e-0cdf604d770b.wav positive
260ef2a5-b9d2-41e7-afc7-bed26e7af5b9.wav positive
a51c6404-cba7-4389-bd37-9a9b01ad4976.wav positive
b6e2e320-f7a4-44d4-9132-ca70f786455d.wav positive
5e117d03-3fcb-41a8-bc6c-2855e9a695fe.wav positive
e160b3c9-c859-4856-9c2a-23707b65fff9.wav positive
799ef10e-7b4e-482b-8ae2-87970f4aa3f7.wav positive


ed988dfa-0bc8-409d-adea-4b44350ad946.wav positive
a0bd7465-ae76-40cd-903b-3591ed9595db.wav positive
cc999a21-8602-4f04-abf9-a86fb36e3d82.wav positive
413931d3-6563-4512-a89c-b749d635b517.wav positive
9b1c4078-453a-42e9-8c5d-beae21d02fd8.wav positive
ee42cab8-6560-49b4-bb5e-bdeaa8f99751.wav positive
c55c80cd-16cd-46ba-9da5-36165ba36c81.wav positive
39e4cc19-0db4-492a-b580-9fc4e4ab62ea.wav positive
16f9b161-86c6-4af1-99cb-0ab327f10fed.wav positive
a580b098-2fe2-40a9-98bf-ae623ec1cd9c.wav positive
2de3fb8f-20a6-4fa5-8428-11cad3338d37.wav positive
a38390a1-3293-4ec3-a17e-cf2776ae2358.wav positive
cbc44a45-447f-4143-b0eb-1bb44185b99a.wav positive
b87bc9b0-61d3-49af-8765-9caaefea55c1.wav positive
1c21cb68-6ea8-4aa1-af22-2e64007554d7.wav positive
1b9fe355-7a78-429a-9e75-a1c547506b2d.wav positive
187c6b84-2999-4ba8-8fe4-3b1936d10f22.wav positive
f86a7ab1-95d7-491a-953a-558877691ecf.wav positive
train_265.wav negative
train_170.wav negative
train_183.wav negative
train_237.wav negative
train_19

In [289]:
np.save(os.path.join('data',exp,'img_train.npy'),img_train)
np.save(os.path.join('data',exp,'label_train.npy'),label_train)

In [290]:
img_train = np.load(os.path.join('data',exp,'img_train.npy'))
label_train = np.load(os.path.join('data',exp,'label_train.npy'))

In [291]:
label_train.shape

(3524,)

## Extracting Devel Data Feature

In [292]:
df_devel = pd.read_csv('csv_files/experiment_data/devel.csv')
df_devel.head()

,filename,label
0,ir3jqGSKT2ManVTAAzj6Ew7orBD2_heavy.wav,positive
1,ZzKpUtFGzsVEK7E1wsVGRftKdx93.wav,positive
2,TkOj2GRygbe2J9HxfMWk02A4e7v1.wav,positive
3,ottC8aetxhRQfKD00RCJc5LWS622_heavy.wav,positive
4,IFMid2LmeCTSQAFaL8wTSUg20No2.wav,positive


In [293]:
img_devel = []
label_devel = []

to_hot_one = {'positive':0, 'negative':1}

for index,row in df_devel.iterrows():
    name = row['filename']
    label = row['label']
    filename = os.path.join(dataset,name)
    
    try:
        print(name, label)
        sound, sr = lb.load(filename, sr=None)
        sound = lb.util.pad_center(sound, 30*sr)
        sound_aug = additive_noise(noisetypes, noiselist, noisesnr, numnoise, audio=sound, noisecat=None)
        
        concat,concat_norm = normalization(sound,sr,scaler)
        img_devel.append(concat_norm)
        label_devel.append(to_hot_one[label])
        
        concat_aug,concat_aug_norm = normalization(sound_aug,sr,scaler)
        img_devel.append(concat_aug_norm)
        label_devel.append(to_hot_one[label])  
    except:
        print('None')

ir3jqGSKT2ManVTAAzj6Ew7orBD2_heavy.wav positive
ZzKpUtFGzsVEK7E1wsVGRftKdx93.wav positive
TkOj2GRygbe2J9HxfMWk02A4e7v1.wav positive
ottC8aetxhRQfKD00RCJc5LWS622_heavy.wav positive
IFMid2LmeCTSQAFaL8wTSUg20No2.wav positive
xYCukI9rgMeslwjIYWheFtyZ6zk1_heavy.wav positive
Iz9X0QRimVT2R7AGhD6jZM0ryPp1.wav positive
CGXh77ubexNuv0UUmOHVCnAgurY2.wav positive
BDygu23lyUbBq2NlqfzRxGoMm9B2_heavy.wav positive
cd57GKgW74bBSqmusnCkc8bXujs2.wav positive
2lsppspH44WH3Scc1ojisHaXJzp1_heavy.wav positive
Hge7rEPBdLcy4ckCtvUFfl4p0Ne2_heavy.wav positive
EUjlCZyR0xMbOH31Je8knI3UdKO2.wav positive
LF7wSEDpWafkz5bwtxnFuho9iWz1.wav positive
y0TkG5sfyhadW3VSct50RTl3HN02.wav positive
t5wo2qvXdBehDj8JuF9pGWbb3Rt1.wav positive
rB4ewmXSc9P5IPhLffnBasjYU1s2.wav positive
h4T9i8JwCEbfw4hzNowalt393Wp1_heavy.wav positive
rAWA4P9I7iYPCIvEheITe8FCRKP2_heavy.wav positive
BRVYzpZ9X1TGGLnjYEHIn3VEJ5j1_heavy.wav positive
lVHnQ9UNkAYmjNm7wL885LzpsOk2_heavy.wav positive
pWFMPFBys1bBerYz5Si4Gb8brGn1.wav positive
Z8ulmLPAsmhYfYuq

/home/viblab/.local/lib/python3.9/site-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")
/home/viblab/.local/lib/python3.9/site-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")
/home/viblab/.local/lib/python3.9/site-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")
/home/viblab/.local/lib/python3.9/site-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")


vXpopcJJEugOInwUtJkEnHgoVhk1.wav positive
IgwjLDmZRyTv03Mq7VdUYL6O5fV2.wav positive
Fh3hBZZRh2UXXicTNGMei8ikbSu2_heavy.wav positive
uRtWGtWi3eVHJ0JhF0ycETkX0ms2.wav positive
4qLi2eWkYGasKhmmPPiyFAGUqBr2_heavy.wav positive
xyNunpsL01N3hkRYCo7pfIkSDgf2_heavy.wav positive
x8PvE190nSZUtv8raWxfg76ek6p2_heavy.wav positive
gpreuyMvtSa8HJhGaliJ6E1Hwx53_heavy.wav positive
tdqx6pwgAaeiofZLQHT82VZcUf33.wav positive
ptuwF1pj4ibIoVuZmt4nKb5naiZ2_heavy.wav positive
0pqMnxeSybfQnGV9nHW3P69FSzE3_heavy.wav positive
W9VOaBOebvQ2uxduqV8wZF4ue572.wav positive
RtRqhfGUUffisj4mpxA9nTYe2cw2.wav positive
YE3Hs5yD9BeIzT8cctZEBKGJIXN2.wav positive


/home/viblab/.local/lib/python3.9/site-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")
/home/viblab/.local/lib/python3.9/site-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")


HATVkqrdXhV2eIiwy5iIP7V0bjo2_heavy.wav positive
5SHNxnq02ygR1L7LRTR3gASXpHb2_heavy.wav positive
8PvWScAB6UbnfrkQrpbqLpu7O0z1_heavy.wav positive
RUnaVFVJiLh22WD0dBl3TeJgmMI3_heavy.wav positive
H9aG9NV6Wae3ATv7xapsyAYdIJy2.wav positive
AlM8enXWaFZ96vN2KM98agcXJvx2.wav positive
euld60aqc2OtnNoKYx9P4WDzVM92.wav positive
iYM9uJMyO1UoXTo1tc7rXvsI4bm1_heavy.wav positive
3qA9Ym3caTbMsegKE0yUvLdapHi1_heavy.wav positive
W9VOaBOebvQ2uxduqV8wZF4ue572_heavy.wav positive
Tvdak0zflIVWdvWsNj7l9Ur0k3m1.wav positive
gxbBHDQWiSWdAnKyqT0BMuH9T2k1.wav positive
0Js6ZUZQ9NUnu568Fh7B6mZ1R8o1.wav positive


...

CUCqq4q5SQa3fDkpraekej7fNza2.wav positive
S0ZhYGoCVfXWMHzPGZ41PvHYiw12.wav positive
rR7IyyhZigMCIvfd2q2cjaGAIXY2_heavy.wav positive
6kMlitGRc3XF2Nt13Wl3LOs4wMl1_heavy.wav positive
aDAVgxAXQOdOakIroRdHwpL867v2.wav positive
H9aG9NV6Wae3ATv7xapsyAYdIJy2_heavy.wav positive
i0EyWP227GaL7ZQRqCS0iysTxcf2_heavy.wav positive
RUZ93kkyD0hJ1v7awXutr6RgF943_heavy.wav positive
0nP3dtfFxhMihttAMq1eoho5KaU2.wav positive
E3nlERKGBPcSO2PjEDJ6GrRKiy73_heavy.wav positive
IjI3eQUHkrYdJSB4nWeXeNoryNl2.wav positive
OUFazbHbbtRTQcRP5BWBKzeZpvw1_heavy.wav positive
txGfgoOqEuPeMPnrjxaRU8i7XjU2.wav positive
G5wWAa7nvkOR9QvfsAt7tsd5qa82.wav positive
2tjg2NfgJPaYdgTZRDUBhWm8F133.wav positive
IFMid2LmeCTSQAFaL8wTSUg20No2_heavy.wav positive
C1h80BY2J5eZgP3UMb4PTJXyKWy2.wav positive
EuEoeANAhadcy5vzBikn3LxlGKh1_heavy.wav positive
6T43bddKoKfG7MwnJWvrPZSsyrc2.wav positive
BDygu23lyUbBq2NlqfzRxGoMm9B2.wav positive
lpohIzcDNEg3DR38ku43WHiCkC42_heavy.wav positive
7lfdqN3JS8ZUOw42lMsIHbyCajA3_heavy.wav positive
B3QbrMswB6

/home/viblab/.local/lib/python3.9/site-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")
/home/viblab/.local/lib/python3.9/site-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")
/home/viblab/.local/lib/python3.9/site-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")
/home/viblab/.local/lib/python3.9/site-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")


BG1rEPoPYKUAHCYVzPrcB3I6VUn2.wav positive
2c70a831-dc41-4d4b-85e8-07c4499b54fe.wav positive
bc4e7e90-4dd2-45f6-a2a4-2c5df914000d.wav positive
936268f0-41a3-421d-bd18-8e69e597e84c.wav positive
4fc9a8e3-909a-4e92-9621-d5e578817f21.wav positive
03c1afa7-eee5-4c18-bd40-c4d91e6d320e.wav positive
00e0a33c-6561-406e-b543-4c9f07b860f7.wav positive
d7d929b4-8ae6-43b9-9328-2e4067de9b17.wav positive
6d194cdc-ba5a-40ba-8232-5bf150026c67.wav positive
ca4579a8-d65b-42aa-8fdf-1108e3b0323e.wav positive
9ced5c4c-9dc3-44f0-96e7-d6b34f8716fd.wav positive
d5a1411d-d312-44b1-a37d-a387ec13df1a.wav positive
7b64a96d-d5f5-4cdf-a020-c71f0c219086.wav positive
ef86e9d1-450c-4df1-8018-038459ac4c14.wav positive
9e4f4e7b-541e-4e88-80c8-458f62c12825.wav positive
20638b85-0ab9-45cc-b257-9ce9986946de.wav positive
3e0fc549-5b05-4405-bfd4-051e7fba7d17.wav positive
126b940c-b6c6-4d3d-adc2-92aafab911b4.wav positive
f7f2f111-2861-4eda-a25e-ec6c90b87c2a.wav positive
a12a101c-c947-42ff-b6b6-57852720e82e.wav positive
04dc09f2

/home/viblab/.local/lib/python3.9/site-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")


116a0485-7052-44f4-9c7f-b367ef7dba89.wav positive
0177b817-0ba3-498a-bc05-48bc66cc91b8.wav positive
36090427-bc31-487b-ae27-1b7105ce39ca.wav positive
d57f155d-09c7-4c77-a482-809eaa23122f.wav positive
9e0490ff-2444-488a-8c01-400b958c9875.wav positive
001d8e33-a4af-4edb-98ba-b03f891d9a6c.wav positive
4f6c6fa4-e708-4c9a-ac8a-7137a4a60093.wav positive
814a5c9e-6174-4efa-a378-a990ac566ae2.wav positive
f4024315-2f10-4142-bd86-8580cf43ac63.wav positive
920ae37e-af2e-4e43-a80e-2150a66db6b1.wav positive
493632f2-537a-4a36-a932-268a8aa59486.wav positive
e4739ce1-f3e5-4af2-adb5-98f022c50bf7.wav positive
f7c229bd-ed49-40d5-a1bc-837ea7b66bd0.wav positive
812a0f06-9e05-459e-84fc-2654bbb8bf9c.wav positive
e3bc1069-6c55-448f-8d73-8d6b8aaa715a.wav positive
13a65aef-e26d-4748-be3f-df3cdb226068.wav positive
7465377a-8963-4a0e-bd1b-92a8924b1ccb.wav positive
ef3f4a93-0a99-4361-8132-5b41fa26610c.wav positive
e67cd4b2-b2c1-4654-b5cb-bbff69a5cf86.wav positive
182246b0-9f77-4c40-976e-3c9342619819.wav positive


In [294]:
np.save(os.path.join('data',exp,'img_devel.npy'),img_devel)
np.save(os.path.join('data',exp,'label_devel.npy'),label_devel)

In [295]:
img_devel = np.load(os.path.join('data',exp,'img_devel.npy'))
label_devel = np.load(os.path.join('data',exp,'label_devel.npy'))

## Extracting Test Data Feature 

In [296]:
df_test = pd.read_csv('csv_files/experiment_data/test.csv')
df_test.head()

,filename,label
0,test_011.wav,positive
1,test_108.wav,positive
2,test_087.wav,negative
3,test_101.wav,negative
4,test_052.wav,positive


In [297]:
img_test = []
label_test = []

to_hot_one = {'positive':0, 'negative':1}

for index,row in df_test.iterrows():
    name = row['filename']
    label = row['label']
    filename = os.path.join(dataset,name)
    
    try:
        print(name, label)
        sound, sr = lb.load(filename, sr=None)
        sound = lb.util.pad_center(sound, 30*sr)
        
        concat,concat_norm = normalization(sound,sr,scaler)
        img_test.append(concat_norm)
        label_test.append(to_hot_one[label])
    except:
        print('None')

test_011.wav positive
test_108.wav positive
test_087.wav negative
test_101.wav negative
test_052.wav positive
test_059.wav negative
test_162.wav negative
test_071.wav negative
test_133.wav negative
test_002.wav negative
test_067.wav positive
test_175.wav negative
test_054.wav negative
test_093.wav positive
test_149.wav negative
test_122.wav negative
test_028.wav negative
test_017.wav negative
test_158.wav negative
test_088.wav negative
test_053.wav negative
test_192.wav negative
test_197.wav positive
test_068.wav negative
test_111.wav negative
test_085.wav positive
test_132.wav negative
test_145.wav negative
test_144.wav negative
test_128.wav negative
test_180.wav negative
test_058.wav positive
test_091.wav negative
test_018.wav negative
test_203.wav negative
test_165.wav negative
test_102.wav positive
test_185.wav negative
test_039.wav positive
test_036.wav positive
test_138.wav negative
test_010.wav negative
test_184.wav positive
test_034.wav negative
test_063.wav negative
test_082.w

In [298]:
np.save(os.path.join('data',exp,'img_test.npy'),img_test)
np.save(os.path.join('data',exp,'label_test.npy'),label_test)

In [299]:
img_test = np.load(os.path.join('data',exp,'img_test.npy'))
label_test = np.load(os.path.join('data',exp,'label_test.npy'))

## Load Data

In [300]:
img_train = np.load(os.path.join('data',exp,'img_train.npy'))
label_train = np.load(os.path.join('data',exp,'label_train.npy'))

img_devel = np.load(os.path.join('data',exp,'img_devel.npy'))
label_devel = np.load(os.path.join('data',exp,'label_devel.npy'))

img_test = np.load(os.path.join('data',exp,'img_test.npy'))
label_test = np.load(os.path.join('data',exp,'label_test.npy'))

In [301]:
img_train = np.array(img_train)
label_train = np.array(label_train)

img_devel = np.array(img_devel)
label_devel = np.array(label_devel)

img_test = np.array(img_test)
label_test = np.array(label_test)

In [302]:
print(img_train.shape)
print(img_devel.shape)
print(img_test.shape)

(3524, 193, 1)
(1198, 193, 1)
(208, 193, 1)


In [303]:
if os.path.exists(os.path.join('data',exp,'array'))==False:
    os.mkdir(os.path.join('data',exp,'array'))

## Save data array

In [304]:
np.save(os.path.join('data',exp,'array','img_train.npy'),img_train)
np.save(os.path.join('data',exp,'array','label_train.npy'),label_train)

np.save(os.path.join('data',exp,'array','img_devel.npy'),img_devel)
np.save(os.path.join('data',exp,'array','label_devel.npy'),label_devel)

np.save(os.path.join('data',exp,'array','img_test.npy'),img_test)
np.save(os.path.join('data',exp,'array','label_test.npy'),label_test)

In [305]:
img_train = np.load(os.path.join('data',exp,'array','img_train.npy'))
label_train = np.load(os.path.join('data',exp,'array','label_train.npy'))

img_val = np.load(os.path.join('data',exp,'array','img_devel.npy'))
label_val = np.load(os.path.join('data',exp,'array','label_devel.npy'))

img_test = np.load(os.path.join('data',exp,'array','img_test.npy'))
label_test = np.load(os.path.join('data',exp,'array','label_test.npy'))

## Augmented using SMOTE

In [306]:
oversample = SMOTE()
img_train_smote, label_train_smote = oversample.fit_resample(np.reshape(img_train,[img_train.shape[0],img_train.shape[1]]), label_train)

In [307]:
oversample = SMOTE()
img_val_smote, label_val_smote = oversample.fit_resample(np.reshape(img_val,[img_val.shape[0],img_val.shape[1]]), label_val)

In [308]:
np.save(os.path.join('data',exp,'array','img_train_smote.npy'),img_train_smote)
np.save(os.path.join('data',exp,'array','label_train_smote.npy'),label_train_smote)

np.save(os.path.join('data',exp,'array','img_devel_smote.npy'),img_val_smote)
np.save(os.path.join('data',exp,'array','label_devel_smote.npy'),label_val_smote)